In [1]:
'''To display density plot of hexagon regimes'''

'''importing libraries'''
import os
import numpy as np
from pathlib import Path
import glob
import pandas
import plotly.express as px
# Import seaborn
import seaborn as sns
import plotly.graph_objects as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

from plotly.subplots import make_subplots
# conus
from scipy import stats  # for sen's slope
import pymannkendall as mk

# define the study period 1988-2022
years = np.arange(1988, 2023)
agent_codes = [1, 2, 3, 4, 5, 6, 7]


In [2]:
'''load landscape-level data'''

## disturbance frequency to show here
# create an avariable to store the informaiton of the disturbance
des       = '/gpfs/sharedfs1/zhulab/Shi/ProjectCONUSDisturbanceAgent/Analysis/V08PostRelease/hexagon_grid_50km_dist_regime_shift_patch_1988_2022'
data_list = sorted(glob.glob(os.path.join(des, 'grid*.csv')))

distdata_grid = []
for data_file in data_list:
    distdata_grid.append(pandas.read_csv(data_file))
distdata_grid = pandas.concat(distdata_grid) 
# replace agent name to display version
distdata_grid.agent[distdata_grid.agent == 'all'] = 'General'
distdata_grid.agent[distdata_grid.agent == 'forest_management'] = 'Logging'
distdata_grid.agent[distdata_grid.agent == 'construction'] = 'Construction'
distdata_grid.agent[distdata_grid.agent == 'agriculture_activity'] = 'Agricultural change'
distdata_grid.agent[distdata_grid.agent == 'stress'] = 'Stress'
distdata_grid.agent[distdata_grid.agent == 'natural_hazard'] = 'Wind/geohazard'
distdata_grid.agent[distdata_grid.agent == 'water_dynamic'] = 'Water dynamic'
distdata_grid.agent[distdata_grid.agent == 'fire'] = 'Fire'
# change MeanY8822 as mean
distdata_grid['regime_mean'] = distdata_grid['MeanY8822']


In [4]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde

# measurements = {'dist_frequency', 'dist_size','dist_size_median', 'dist_size_prct75', 'dist_size_prct99', 'dist_size_max', 'dist_severity', 'dist_grid_area_adjust'};
regime_indicators = {'frequency', 'size', 'size_max', 'severity'}
dist_agents       = {'General'}

regime_indicators = ['frequency', 'size', 'severity']
dist_agents       = ['General', 'Logging', 'Construction', 'Agricultural change', 'Stress', 'Wind/geohazard', 'Fire', 'Water dynamic']

# iterate through the regime indicators
for regime in regime_indicators:

    # landscape grid -level frequency
    regime_name = f'dist_{regime}'
    # select the dist regime for the agent
    distdata_grid_regime_agent = distdata_grid[distdata_grid['measurement'] == regime_name]
    # only the selected agents
    distdata_grid_regime_agent = distdata_grid_regime_agent[distdata_grid_regime_agent['agent'].isin(dist_agents)]

    if regime == 'frequency':
        distdata_grid_regime_agent['regime_mean'] = distdata_grid_regime_agent['regime_mean'] * 100 # to patches/mk2 from patches/ha

    records_attributes = pandas.DataFrame(columns=['agent', 'mean', 'std', 'p0', 'p1', 'p25', 'p50', 'p75', 'p99', 'p100'])
    fig = go.Figure()
    fig = make_subplots(rows=1, cols=8, shared_yaxes=True,)
    
    max_val = distdata_grid_regime_agent['regime_mean'].quantile(1)
    if regime == 'frequency':
        max_val = 1 # patches/mk2
        buf_val = 0.05
        ext_val = max_val*buf_val
        y_max = 98
        # only one color blue
        colors = ['#053061', '#053061',  '#053061', '#053061','#053061' ,  '#053061',  '#053061', '#053061']
    elif regime == 'size':
        max_val = 10 # ha
        buf_val = 0.05
        ext_val = max_val*buf_val
        y_max = 45
        colors = ['#40004b', '#40004b', '#40004b', '#40004b', '#40004b', '#40004b', '#40004b', '#40004b', '#40004b', '#40004b']
    elif regime == 'severity':
        max_val == 4
        buf_val = 0.05
        ext_val = max_val*buf_val
        y_max = 45
        # use blind safe colors
        colors = ['#67001f', '#67001f', '#67001f', '#67001f', '#67001f', '#67001f', '#67001f', '#67001f', '#67001f', '#67001f']
    # loop each of the agents
    for i, agt in enumerate(dist_agents):
        # select the data for the agent
        agt_data = distdata_grid_regime_agent[distdata_grid_regime_agent['agent'] == agt]
        # fill the nan values with 0, as we are counting all grid cells over CONUS
        agt_data = agt_data.fillna(0) # fill nan with zero for the grid
        agt_data = agt_data['regime_mean']
        # print the total number of the grids
        print(f'Total number of the grids for {agt} is {len(agt_data)}')
        
        _mean = agt_data.mean()
        # record the percentiles 0, 1, 25, 50, 75, 99, 100 of the data
        percts = agt_data.quantile([0, 0.01, 0.25, 0.5, 0.75, 0.99, 1])
        # add to the records_patch_attributes by dataframe concatenation
        records_attributes = pandas.concat([records_attributes, pandas.DataFrame({
            'agent': [agt],
            'mean': [_mean],
            'std': [agt_data.std()],
            'p0': [percts[0]],
            'p1': [percts[0.01]],
            'p25': [percts[0.25]],
            'p50': [percts[0.5]],
            'p75': [percts[0.75]],
            'p99': [percts[0.99]],
            'p100': [percts[1]]
        })])
          
        _agt_data = agt_data.copy()
        _agt_data[_agt_data > max_val] = max_val + buf_val - 0.0001 # to make sure the last bin is not empty
        fig.add_trace(go.Histogram(
            x=_agt_data,
            histnorm='percent',
            name=agt, # name used in legend and hover labels
            xbins=dict( # bins used for histogram
                start=0,
                end=max_val + buf_val,
                size=max_val*buf_val
            ),
            marker_color=colors[i],
            # opacity=0.75
        ),row=1, col=i+1)
        
        # add number to the line
        _xshift = 50
        _mean = agt_data.mean()
        if _mean < 0.01:
            _mean=f'Mean:{_mean:.3f}'
        else:
            _mean=f'Mean:{_mean:.2f}'
        _median = agt_data.median()
        if _median < 0.01:
            _median=f'Median:{_median:.3f}'
        else:
            _median=f'Median:{_median:.2f}'
            
        fig.add_annotation(
            x=0,
            y=y_max-20,
            text=_mean,
            showarrow=False,
            font=dict(
                color=colors[i],
            ),
            xshift=_xshift,
            yshift=0,
            row=1, col=i+1
        )
        # same x range
        fig.update_xaxes(range=[0, max_val + ext_val], row=1, col=i+1)
        # update y range 0-50%
        fig.update_yaxes(range=[0, y_max], row=1, col=i+1)
        
    # adjust size
    fig.update_layout(
        template='simple_white',
        width=900, height=180,
        # xaxis_title=regime,
        # yaxis_title="Landscapes (%)",
        hovermode="x",
        bargap=0,
        margin=dict(l=0, r=4, t=0, b=0),
    )
    # do not display legend
    fig.update_layout(showlegend=False)
    fig.show()
    # save as svg
    fig.write_image(f'hexagon_hist_{regime}.svg')
    print(records_attributes)


    

Total number of the grids for General is 3391
Total number of the grids for Logging is 3391
Total number of the grids for Construction is 3391
Total number of the grids for Agricultural change is 3391
Total number of the grids for Stress is 3391
Total number of the grids for Wind/geohazard is 3391
Total number of the grids for Fire is 3391
Total number of the grids for Water dynamic is 3391


/tmp/ipykernel_2692163/2393609660.py:64: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



                 agent      mean       std   p0        p1       p25       p50  \
0              General  0.362244  0.405495  0.0  0.002338  0.113953  0.236115   
0              Logging  0.088122  0.163238  0.0  0.000000  0.000446  0.009812   
0         Construction  0.051101  0.116680  0.0  0.000011  0.001924  0.012439   
0  Agricultural change  0.132023  0.217350  0.0  0.000000  0.009264  0.054404   
0               Stress  0.008287  0.023023  0.0  0.000000  0.000034  0.000388   
0       Wind/geohazard  0.003982  0.023880  0.0  0.000000  0.000000  0.000023   
0                 Fire  0.005955  0.018910  0.0  0.000000  0.000000  0.000034   
0        Water dynamic  0.072775  0.277588  0.0  0.000046  0.005891  0.019087   

        p75       p99       p100  
0  0.489608  1.629223  10.689990  
0  0.088872  0.705507   0.899001  
0  0.047275  0.585560   1.722719  
0  0.152085  1.069487   1.888363  
0  0.004297  0.124965   0.249217  
0  0.000263  0.081040   0.566925  
0  0.001726  0.091881   0

/tmp/ipykernel_2692163/2393609660.py:64: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



                 agent         mean           std   p0        p1       p25  \
0              General    66.364831   1042.714466  0.0  0.764785  1.307269   
0              Logging     2.161049      4.518983  0.0  0.000000  0.832500   
0         Construction     1.516915      2.677266  0.0  0.360000  0.861391   
0  Agricultural change     1.561948      3.267226  0.0  0.000000  1.001554   
0               Stress  1482.799866  11638.569664  0.0  0.000000  0.472500   
0       Wind/geohazard     5.161085     29.705082  0.0  0.000000  0.000000   
0                 Fire    56.588581    268.309715  0.0  0.000000  0.000000   
0        Water dynamic     1.527162      3.294931  0.0  0.360000  0.841159   

        p50        p75           p99           p100  
0  1.916085   3.170839    990.495815   39358.299130  
0  1.378983   2.406110     17.683350      97.434000  
0  1.083838   1.424775      9.175280      80.362500  
0  1.260753   1.643310      5.807472     152.927143  
0  0.729643   1.531508  443

/tmp/ipykernel_2692163/2393609660.py:64: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



                 agent      mean       std   p0        p1       p25       p50  \
0              General  2.217727  0.429119  0.0  1.276123  1.943217  2.220934   
0              Logging  1.916942  0.735786  0.0  0.000000  1.579836  2.028254   
0         Construction  2.310449  0.488054  0.0  1.000000  2.068199  2.355224   
0  Agricultural change  2.238692  0.544025  0.0  0.000000  1.937990  2.242810   
0               Stress  1.655297  0.983217  0.0  0.000000  1.064728  1.557377   
0       Wind/geohazard  1.604901  1.460106  0.0  0.000000  0.000000  1.982090   
0                 Fire  1.364798  1.307806  0.0  0.000000  0.000000  1.567839   
0        Water dynamic  2.366531  0.519817  0.0  1.172266  2.031186  2.388134   

        p75       p99      p100  
0  2.510649  3.245566  3.780894  
0  2.362382  3.448718  4.000000  
0  2.576733  3.434594  4.000000  
0  2.601414  3.333333  4.000000  
0  2.415230  4.000000  4.000000  
0  2.932913  4.000000  4.000000  
0  2.438048  4.000000  4.000000 